In [1]:
def Case(OrderSize, Width, Length, Weight):
    Model = pyo.ConcreteModel(name = ModelName + ', Order size ' + str(OrderSize))
    Model.Engine = SolverName
    Model.TimeLimit = TimeLimit
    Solver, Model = SetUpSolver(Model)
    DefineModelData(Model, Width, Length, Weight)
    Model.Orders = OrderSize
    DefineModel(Model)
    WriteModelToFile(WriteFile, Model)
    Results = CallSolver(Solver, Model)
    Obj = pyo.value(Model.Obj())
    Products = ''
    for c in Model.Candidate:
        if np.isclose(pyo.value(Model.Select[c]), 1):   # Binary variable = 1, give-or-take small precision error
            Products += str(pyo.value(Model.CandidateWidth[c])).rjust(4) + ' ' + str(pyo.value(Model.CandidateLength[c])).rjust(4) + ' '
    return Obj, Products, pyo.value(Model.Baseline)

In [2]:
def OuterLoop(Size, BestFound):   # Solve each product size multiple times, to increase chance of getting a good solution
    Width, Length, Weight = GetData(DataFile, DataWorksheet)
    for OrderSize in range(Size, Size + 1):
        Result, Products, Baseline = Case(OrderSize, Width, Length, Weight)
        Marker = ' '
        if Result < BestFound:
            BestFound = Result
            Marker = '*'   # Mark each solution that is the best so far, for this number of products
        Waste = Result / Baseline
        WriteOutput(OrderSize, Result, Waste, Marker, Products)
    return BestFound

In [3]:
def Main():
    Timer('Start');
    print(ModelName)
    print('Data file:', DataFile, '\n')
    print('Products  Objective      Waste      Sizes')
    for s in range(ProductsMin, ProductsMax + 1):   # Run multiple product cases, if required
        BestFound = np.inf
        for i in range(0, Iterations):
            BestFound = OuterLoop(s, BestFound)
        print()   # Blank line between each set of product iterations
        Timer('Completed ' + str(s));
    Timer('Finish');
    WriteCheckpoints()